In [2]:
import pandas as pd 
from datetime import datetime


In [3]:
df=pd.read_csv('../Data/dataset.csv',encoding='ISO-8859-1')

In [4]:
pd.set_option('display.max_columns',None)

In [5]:
df_100=df.iloc[:1000,:]

In [6]:
df_100.drop(columns=['Product Status','Product Image','Product Description','Customer Email'],inplace=True)

C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\692516603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100.drop(columns=['Product Status','Product Image','Product Description','Customer Email'],inplace=True)


### Order Characteristics

In [7]:
# Convert 'order date (DateOrders)' to datetime, using the correct format string
df_100['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'], format="%m/%d/%Y %H:%M")

# Define cutoff time at 5:00 PM on each day
df_100['Cutoff Time'] = df_100['order date (DateOrders)'].dt.date.astype('datetime64[ns]') + pd.Timedelta(hours=17)

# Calculate the difference in hours between Cutoff Time and Order Time
df_100['Time to Cutoff (hours)'] = (df_100['Cutoff Time'] - df_100['order date (DateOrders)']).dt.total_seconds() / 3600

C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\707606146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'], format="%m/%d/%Y %H:%M")
C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\707606146.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['Cutoff Time'] = df_100['order date (DateOrders)'].dt.date.astype('datetime64[ns]') + pd.Timedelta(hours=17)
C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\707606

In [8]:
df_100['shipping date (DateOrders)']=pd.to_datetime(df_100['shipping date (DateOrders)'])

C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\2274148962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['shipping date (DateOrders)']=pd.to_datetime(df_100['shipping date (DateOrders)'])


In [10]:
df_100['Multi-item Flag']=df_100['Order Item Quantity'].apply(lambda x: 1 if x==1 else 0)

C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\1591656606.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['Multi-item Flag']=df_100['Order Item Quantity'].apply(lambda x: 1 if x==1 else 0)


In [11]:
df_100[['Late_delivery_risk','Multi-item Flag','Time to Cutoff (hours)']].corr()

,Late_delivery_risk,Multi-item Flag,Time to Cutoff (hours)
Late_delivery_risk,1.000000,0.160334,-0.015333
Multi-item Flag,0.160334,1.000000,-0.027270
Time to Cutoff (hours),-0.015333,-0.027270,1.000000


In [12]:
df_100['Delivery Time']=df_100['shipping date (DateOrders)']-df_100['order date (DateOrders)']
df_100['Delivery Time']=df_100['Delivery Time'].astype(str)
df_100['Delivery Time']=df_100['Delivery Time'].str[0]
df_100['Delivery Time']=df_100['Delivery Time'].astype(int)

C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\1190356437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['Delivery Time']=df_100['shipping date (DateOrders)']-df_100['order date (DateOrders)']
C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\1190356437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['Delivery Time']=df_100['Delivery Time'].astype(str)
C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\1190356437.py:3: SettingWithCopyWarning: 
A value is trying to be set on

### Customer Average Delivery Time:

In [26]:
cus_avg_delivery_time=df_100[['Shipping Mode','Delivery Time']].groupby(['Shipping Mode']).mean().to_dict()

In [27]:
cus_avg_delivery_time['Delivery Time']

{'First Class': 2.0,
 'Same Day': 0.0,
 'Second Class': 4.166666666666667,
 'Standard Class': 3.919575113808801}

In [28]:
df_100['Shipp_avg_delivery_time']=df_100['Shipping Mode'].map(cus_avg_delivery_time['Delivery Time'])

C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\1867257827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['Shipp_avg_delivery_time']=df_100['Shipping Mode'].map(cus_avg_delivery_time['Delivery Time'])


In [15]:
df_100['cus_avg_delivery_time']=df_100['Customer Id'].map(cus_avg_delivery_time['Delivery Time'])

C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\2825195852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['cus_avg_delivery_time']=df_100['Customer Id'].map(cus_avg_delivery_time['Delivery Time'])


In [29]:
df_100[['Shipp_avg_delivery_time','Late_delivery_risk','Delivery Time']].corr()

,Shipp_avg_delivery_time,Late_delivery_risk,Delivery Time
Shipp_avg_delivery_time,1.000000,-0.012419,0.402444
Late_delivery_risk,-0.012419,1.000000,0.574880
Delivery Time,0.402444,0.574880,1.000000


### Regional Delay Rate:

In [17]:
total_deliveries_each_region=df_100[['Order Region','Late_delivery_risk']].groupby('Order Region').count().to_dict()

In [18]:
late_total_deliveries_each_region=df_100.loc[df_100['Late_delivery_risk']==1,['Order Region','Late_delivery_risk']].groupby('Order Region').count().to_dict()

In [19]:
df_100['Regional_Delivery_Rate']=df_100['Order Region'].apply(lambda x:late_total_deliveries_each_region['Late_delivery_risk'][x]/total_deliveries_each_region['Late_delivery_risk'][x] if x in late_total_deliveries_each_region['Late_delivery_risk'].keys() else 0)

C:\Users\10139519\AppData\Local\Temp\ipykernel_25388\541599187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['Regional_Delivery_Rate']=df_100['Order Region'].apply(lambda x:late_total_deliveries_each_region['Late_delivery_risk'][x]/total_deliveries_each_region['Late_delivery_risk'][x] if x in late_total_deliveries_each_region['Late_delivery_risk'].keys() else 0)


In [20]:
df_100[['cus_avg_delivery_time','Regional_Delivery_Rate','Late_delivery_risk']].corr()

,cus_avg_delivery_time,Regional_Delivery_Rate,Late_delivery_risk
cus_avg_delivery_time,1.000000,0.129754,0.555781
Regional_Delivery_Rate,0.129754,1.000000,0.227260
Late_delivery_risk,0.555781,0.227260,1.000000


### Product-Specific Attributes

In [ ]:
from transformers import pipeline

# This model is a `zero-shot-classification` model.
# It will classify text, except you are free to choose any label you might imagine
classifier = pipeline(model="facebook/bart-large-mnli")

In [30]:
def classify(product):
    res=classifier(
        "Define these product in which classification it goes 'Titleist Pro V1 High Numbers Personalized Gol'",
        candidate_labels=["Fragility", "Hazardous", "Normal"],
    )
    return res



In [31]:
product_classification=df_100['Product Name'].apply(lambda x:classify(x))

In [37]:
df_100.tail()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Name,Product Price,shipping date (DateOrders),Shipping Mode,Cutoff Time,Time to Cutoff (hours),Multi-item Flag,Delivery Time,cus_avg_delivery_time,Regional_Delivery_Rate
995,DEBIT,3,4,120.940002,251.960007,Advance shipping,0,17,Cleats,Encinitas,EE. UU.,Ryan,11510,Smith,XXXXXXXXX,Consumer,CA,5042 Cozy Acres,92024.0,4,Apparel,33.046471,-117.289673,Europe,Bognor Regis,Reino Unido,11510,2017-07-23 02:53:00,63992,365,47.990002,0.16,159991,59.990002,0.48,5,299.950012,251.960007,120.940002,Northern Europe,Inglaterra,COMPLETE,NaN,365,17,Perfect Fitness Perfect Rip Deck,59.990002,2017-07-26 02:53:00,Standard Class,2017-07-23 17:00:00,14.116667,0,3,3.0,0.701299
996,DEBIT,5,4,-56.689999,251.960007,Late delivery,1,17,Cleats,Florissant,EE. UU.,Mary,9500,Vance,XXXXXXXXX,Consumer,MO,4466 Little Grove,63031.0,4,Apparel,38.796600,-90.346405,Europe,Waterlooville,Reino Unido,9500,2015-09-05 16:38:00,16969,365,47.990002,0.16,42426,59.990002,-0.23,5,299.950012,251.960007,-56.689999,Northern Europe,Inglaterra,COMPLETE,NaN,365,17,Perfect Fitness Perfect Rip Deck,59.990002,2015-09-10 16:38:00,Standard Class,2015-09-05 17:00:00,0.366667,0,5,5.0,0.701299
997,DEBIT,6,4,4.980000,248.960007,Late delivery,1,17,Cleats,Toms River,EE. UU.,Mary,10821,Cooper,XXXXXXXXX,Consumer,NJ,112 Green Fawn Estates,8753.0,4,Apparel,40.040562,-74.158875,Europe,Valencia,España,10821,2015-09-22 11:53:00,18120,365,50.990002,0.17,45281,59.990002,0.02,5,299.950012,248.960007,4.980000,Southern Europe,Valenciana,COMPLETE,NaN,365,17,Perfect Fitness Perfect Rip Deck,59.990002,2015-09-28 11:53:00,Standard Class,2015-09-22 17:00:00,5.116667,0,6,6.0,0.462687
998,DEBIT,5,4,21.910000,248.960007,Late delivery,1,17,Cleats,Honolulu,EE. UU.,Amanda,4727,Estes,XXXXXXXXX,Consumer,HI,9254 Thunder Meadow,96822.0,4,Apparel,21.305620,-157.838699,Europe,Biarritz,Francia,4727,2015-08-01 07:03:00,14544,365,50.990002,0.17,36384,59.990002,0.09,5,299.950012,248.960007,21.910000,Western Europe,Aquitania-Lemosín-Poitou-Charentes,COMPLETE,NaN,365,17,Perfect Fitness Perfect Rip Deck,59.990002,2015-08-06 07:03:00,Standard Class,2015-08-01 17:00:00,9.950000,0,5,5.0,0.635294
999,DEBIT,2,4,44.810001,248.960007,Advance shipping,0,17,Cleats,Chicago,EE. UU.,Amy,10647,Johnson,XXXXXXXXX,Consumer,IL,2058 Quaking Oak Common,60622.0,4,Apparel,41.896149,-87.679329,Europe,Aix-en-Provence,Francia,10647,2015-07-24 05:19:00,13991,365,50.990002,0.17,35002,59.990002,0.18,5,299.950012,248.960007,44.810001,Western Europe,Provenza-Alpes-Costa Azul,COMPLETE,NaN,365,17,Perfect Fitness Perfect Rip Deck,59.990002,2015-07-26 05:19:00,Standard Class,2015-07-24 17:00:00,11.683333,0,2,2.0,0.635294


In [43]:
product_classification

0      {'sequence': 'Define these product in which cl...
1      {'sequence': 'Define these product in which cl...
2      {'sequence': 'Define these product in which cl...
3      {'sequence': 'Define these product in which cl...
4      {'sequence': 'Define these product in which cl...
                             ...                        
995    {'sequence': 'Define these product in which cl...
996    {'sequence': 'Define these product in which cl...
997    {'sequence': 'Define these product in which cl...
998    {'sequence': 'Define these product in which cl...
999    {'sequence': 'Define these product in which cl...
Name: Product Name, Length: 1000, dtype: object

In [53]:
product_class=[]
for i in range(1000):
    
    product_class.append(product_classification[i]['scores'])

In [ ]:
product_classification[0]['labels']

{'sequence': "Define these product in which classification it goes 'Titleist Pro V1 High Numbers Personalized Gol'",
 'labels': ['Normal', 'Hazardous', 'Fragility'],
 'scores': [0.36616232991218567, 0.3520227372646332, 0.28181490302085876]}

In [54]:
product_class

[[0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.

In [55]:
product_class=pd.DataFrame(product_class,columns=product_classification[0]['labels'])

In [56]:
product_class

,Normal,Hazardous,Fragility
0,0.366162,0.352023,0.281815
1,0.366162,0.352023,0.281815
2,0.366162,0.352023,0.281815
3,0.366162,0.352023,0.281815
4,0.366162,0.352023,0.281815
...,...,...,...
995,0.366162,0.352023,0.281815
996,0.366162,0.352023,0.281815
997,0.366162,0.352023,0.281815
998,0.366162,0.352023,0.281815


In [58]:
df_1000=pd.concat([df_100,product_class],axis=1)

In [61]:
df_1000[product_classification[0]['labels']+['Late_delivery_risk']].corr()

,Normal,Hazardous,Fragility,Late_delivery_risk
Normal,NaN,NaN,NaN,NaN
Hazardous,NaN,NaN,NaN,NaN
Fragility,NaN,NaN,NaN,NaN
Late_delivery_risk,NaN,NaN,NaN,1.0


In [45]:
product_class

[[0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.3520227372646332, 0.28181490302085876],
 [0.36616232991218567, 0.

In [22]:
df_100.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Name,Product Price,shipping date (DateOrders),Shipping Mode,Cutoff Time,Time to Cutoff (hours),Multi-item Flag,Delivery Time,cus_avg_delivery_time,Regional_Delivery_Rate
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.0,2,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,2018-01-31 22:56:00,77202,1360,13.110000,0.04,180517,327.75,0.29,1,327.75,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,Smart watch,327.75,2018-02-03 22:56:00,Standard Class,2018-01-31 17:00:00,-5.933333,1,3,3.0,0.455882
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725.0,2,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,2018-01-13 12:27:00,75939,1360,16.389999,0.05,179254,327.75,-0.80,1,327.75,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,Smart watch,327.75,2018-01-18 12:27:00,Standard Class,2018-01-13 17:00:00,4.550000,1,5,5.0,0.547945
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125.0,2,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,2018-01-13 12:06:00,75938,1360,18.030001,0.06,179253,327.75,-0.80,1,327.75,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,Smart watch,327.75,2018-01-17 12:06:00,Standard Class,2018-01-13 17:00:00,4.900000,1,4,4.0,0.547945
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,EE. UU.,Tana,19490,Tate,XXXXXXXXX,Home Office,CA,3200 Amber Bend,90027.0,2,Fitness,34.125946,-118.291016,Pacific Asia,Townsville,Australia,19490,2018-01-13 11:45:00,75937,1360,22.940001,0.07,179252,327.75,0.08,1,327.75,304.809998,22.860001,Oceania,Queensland,COMPLETE,NaN,1360,73,Smart watch,327.75,2018-01-16 11:45:00,Standard Class,2018-01-13 17:00:00,5.250000,1,3,3.0,0.414634
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,Orli,19489,Hendricks,XXXXXXXXX,Corporate,PR,8671 Iron Anchor Corners,725.0,2,Fitness,18.253769,-66.037048,Pacific Asia,Townsville,Australia,19489,2018-01-13 11:24:00,75936,1360,29.500000,0.09,179251,327.75,0.45,1,327.75,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,Smart watch,327.75,2018-01-15 11:24:00,Standard Class,2018-01-13 17:00:00,5.600000,1,2,2.0,0.414634


In [25]:
df_100['Customer Id'].nunique()

856